In [ ]:
import pandas as pd
import json
from flatten_json import flatten
import requests
import math
from tqdm import tqdm
from time import sleep
# import matplotlib.pyplot as plt
from collections import Counter
import re
# import seaborn as sns
# from matplotlib.dates import MonthLocator
# from matplotlib.ticker import PercentFormatter
import statistics
import numpy as np

## Скачивание вакансий "Работа в России"

In [ ]:
def get_salaries_by_region(**kwargs):
    url = f"http://opendata.trudvsem.ru/api/v1/vacancies"
    
    params = {**kwargs}
    # date_params = {start: "01-01-2019"}
    data = requests.get(url, params).json()
    return data

In [ ]:
region = [
            '0100000000000',
            '0200000000000',
            '0300000000000',
            '0400000000000',
            '0500000000000',
            '0600000000000',
            '0700000000000',
            '0800000000000',
            '0900000000000',
            '1000000000000',
            '1100000000000',
            '1200000000000',
            '1300000000000',
            '1400000000000',
            '1500000000000',
            '1600000000000',
            '1700000000000',
            '1800000000000',
            '1900000000000',
            '2000000000000',
            '2100000000000',
            '2200000000000',
            '2300000000000',
            '2400000000000',
            '2500000000000',
            '2600000000000',
            '2700000000000',
            '2800000000000',
            '2900000000000',
            '3000000000000',
            '3100000000000',
            '3200000000000',
            '3300000000000',
            '3400000000000',
            '3500000000000',
            '3600000000000',
            '3700000000000',
            '3800000000000',
            '3900000000000',
            '4000000000000',
            '4100000000000',
            '4200000000000',
            '4300000000000',
            '4400000000000',
            '4500000000000',
            '4600000000000',
            '4700000000000',
            '4800000000000',
            '4900000000000',
            '5000000000000',
            '5100000000000',
            '5200000000000',
            '5300000000000',
            '5400000000000',
            '5500000000000',
            '5600000000000',
            '5700000000000',
            '5800000000000',
            '5900000000000',
            '6000000000000',
            '6100000000000',
            '6200000000000',
            '6300000000000',
            '6400000000000',
            '6500000000000',
            '6600000000000',
            '6700000000000',
            '6800000000000',
            '6900000000000',
            '7000000000000',
            '7100000000000',
            '7200000000000',
            '7300000000000',
            '7400000000000',
            '7500000000000',
            '7600000000000',
            '7700000000000',
            '7800000000000',
            '7900000000000',
            '8300000000000',
            '8600000000000',
            '8700000000000',
            '8900000000000',
            '9100000000000',
            '9200000000000',
        ]

In [ ]:
# request vacancies in Medicine for each region, starting from 01 Jan 2022 
# calculate an offset for each region and get all of the results for a region, 
# store each request with 100 vacancies titled by the number of the region and sequence number of the offset

for i in tqdm(region):
    page_result = get_salaries_by_region(region_code=i, industry="Medicine", start="01-01-2022", text="акушер")
    total = page_result["meta"]["total"]
    offset = list(range(0,math.ceil(total/100)))
    
    for o in offset:
        page_result = get_salaries_by_region(offset=str(o), limit=100, region_code=i, industry="Medicine", text="акушер")

        with open('../output/API 3.0/'+i+'_'+str(o)+'.json', 'w') as json_file:
            json.dump(page_result, json_file)
        
        sleep(0.0001)

In [ ]:
import os

# a function to get all data from trudvsem API with offset
for i in tqdm(region):
    file_path = '../output/API 3.0/' + i + '_0.json'
    if not os.path.exists(file_path):
        continue
    with open(file_path, 'r') as f:
        json_data = json.loads(f.read())
        total = json_data['meta']['total']
        offset = list(range(0, math.ceil(total / 100)))

        all_units = []

    for o in offset:
        json_file = "../output/API 3.0/" + i + "_" + str(o) + ".json"
        if not os.path.exists(json_file):
            continue
        with open(json_file, 'r') as f:
            json_data = json.loads(f.read())
            vacancies = json_data['results']['vacancies']

        for unit in vacancies:
            flat = flatten(unit)
            all_units.append(flat)
        sleep(0.0001)

    df = pd.DataFrame(all_units)
    df.to_csv('../output/API 3.0/merged_obgyn_' + i + '.csv')
print("all done!")

In [ ]:
merge = pd.read_csv('../output/API 3.0/merged_obgyn_'+str(region[0])+'.csv')

# open each file with region code
for i in tqdm(region[1:]):
    # print(i)
    file_path = '../output/API 3.0/merged_obgyn_' + i + '.csv'
    if not os.path.exists(file_path):
        continue
    df = pd.read_csv(file_path)
    merge = pd.concat([merge, df])

print(merge.iloc[:10, [1,3,5]])

In [ ]:
merge.to_csv('../output/API 3.0/MAIN dataset.csv')

## Анализ вакансий "Работа в России"

### Загрузка и очистка данных

In [ ]:
df = pd.read_csv('../output/API 3.0/MAIN dataset.csv')
df.head(3)

In [ ]:
df.shape

In [ ]:
# to clean up raw dataset
df.columns = [col.replace('vacancy_', '') for col in df.columns]
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
# приводим коды профессий и дату создания вакансии к удобному формату данных
df['code_profession'] = df['code_profession'].astype('string')
df['creation-date']=df['creation-date'].astype('datetime64[ns]')

In [ ]:
df.columns

In [ ]:
df = df[['id', 'region_region_code', 'region_name', 'vac_url', 'creation-date', 'code_profession', 'job-name', 'employment', 'salary_min', 'salary_max', 'duty', 'term_text']]
df.tail(3)

In [ ]:
# for MAIN df w/o personnel
print(f"Кол-во строк и столбцов:{df.shape}", "\nНазвание колонок:\n", df.columns)

In [ ]:
pd.set_option('display.max_rows', 1000)
df.isnull().sum()

In [ ]:
df['region_name'].isna().describe()

In [ ]:
df['code_profession'].value_counts()

In [ ]:
# по коду профессии ОКПДТР удаляем вакансии, которые не относятся к акушерству 
df['code_profession'] = df['code_profession'].astype('string')
irrelevant_codes = ['181121.0', '204751.0', '204613.0', '221047.0', '192588.0', '260215.0', '260732.0']
df = df[~df['code_profession'].isin(irrelevant_codes)]

In [ ]:
# add_updates = df[['id', 'vac_url']].copy()
# add_updates.to_csv('../output/add_updates.csv', index=False)

### Преобразование данных

In [ ]:
# удаляем лишние колонки с индексами и вакансии, размещенные раньше мая 2023 года
# df = df.loc[df['creation-date'] >= '2023-05-01']

In [ ]:
personnel = pd.read_csv('../data/personnel category.csv')
personnel['code_profession'] = personnel['code_profession'].astype('string')

In [ ]:
code_to_personnel = dict(zip(personnel['code_profession'], personnel['вид персонала']))

def assign_personnel(code_profession):
    # Return the personnel category for the given code_profession, or None if not found
    return code_to_personnel.get(code_profession, None)

df['personnel'] = df['code_profession'].apply(assign_personnel)

In [ ]:
df['personnel'].value_counts()

In [ ]:
df.to_excel('../output/API 3.0/MAIN dataset with personnel.xlsx', index=False)

### Осмотр данных (MAIN w/ personnel)

In [ ]:
df = pd.read_excel('../output/API 3.0/MAIN dataset with personnel.xlsx', sheet_name='Sheet1')
df.shape

In [ ]:
df.columns

In [ ]:
df['region_name'].value_counts().to_clipboard()

In [ ]:
# For every unique code_profession, find the most common job-name
most_common_jobname = (
    df.groupby('code_profession')['job-name']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    .reset_index()
    .rename(columns={'job-name': 'most_common_job_name'})
)
most_common_jobname

In [ ]:
# # прочесть описание вакансии по ссылке, используя код ОКПДТР
# pd.set_option('display.max_colwidth', None)
# df.loc[df['code_profession'] == '260732.0', 'vac_url']

#### Кол-во вакансий

In [ ]:
df['year'] = df['creation-date'].dt.year
regions_per_year_personnel = (
    df.groupby(['year', 'personnel'])['region_name']
    .nunique()
    .unstack(fill_value=0)
)
print(regions_per_year_personnel)

In [ ]:
# Group by month and personnel, count number of vacancies
vacancies_by_month_personnel = (
    df_2024_2025.groupby(['month', 'personnel'])['id']
    .count()
    .unstack(fill_value=0)
    .sort_index()
)

print(vacancies_by_month_personnel)


#### Зарплата в целом

In [ ]:
# Группировка по code_profession и расчет средних значений salary_min и salary_max
df.groupby('personnel')[['salary_min', 'salary_max']].mean().loc[
    df['personnel'].value_counts(ascending=False).index
]

In [ ]:
top10_salary_min = df.groupby('region_name')['salary_min'].min().sort_values(ascending=False).tail(10)
top10_salary_min

In [ ]:
# Calculate median salary_min and salary_max for each region and personnel
median_salary_min = df.groupby(['region_name', 'personnel'])['salary_min'].median()
median_salary_max = df.groupby(['region_name', 'personnel'])['salary_max'].median()

# Combine into a DataFrame
median_salaries = pd.DataFrame({
    'median_salary_min': median_salary_min,
    'median_salary_max': median_salary_max
})

# Calculate the average of the two medians for each region and personnel
median_salaries['median_salary_avg'] = median_salaries[['median_salary_min', 'median_salary_max']].mean(axis=1)

# Copy the average median salary to clipboard
median_salaries['median_salary_avg'].to_clipboard()

In [ ]:
median_salary_table = df.groupby(['region_name', 'personnel'])['salary_min'].median().unstack()
median_salary_table.to_clipboard()
# print(median_salary_table)
# median_salary_table.to_excel('../output/salaries_by_region.xlsx')

In [ ]:
df['region_name'].value_counts()

In [ ]:
df[df['personnel'] == 'средний']['region_name'].value_counts(ascending=False)

In [ ]:
# зарплаты ниже мрот 2025 года
pd.set_option('display.max_colwidth', None)
df.loc[df['salary_min'] < 22440, 'vac_url']

In [ ]:
# выделяем из массива данных только вакансии с полной занятостью
# filtered_df = filtered_df[filtered_df['vacancy_employment'] == 'Полная занятость']

In [ ]:
df[(df['salary_min'] <= 22440) & (df['employment'] == 'Полная занятость')]['region_name'].describe()

### Визуализация данных

In [ ]:
# which schools are searching for the majority of vacancies by region
most_frequent_schools = filtered_df.groupby('vacancy_region_name')['company_name_copy'].value_counts().reset_index(name='count')
most_frequent_schools.columns = ['vacancy_region_name', 'Most Frequent School', 'Count']
most_frequent_schools = most_frequent_schools.sort_values(by='Count', ascending=False)
most_frequent_schools = most_frequent_schools.drop_duplicates(subset='vacancy_region_name')
total_vacancies = filtered_df.groupby('vacancy_region_name')['vacancy_id'].count().reset_index(name='Total Vacancies')
most_frequent_schools = most_frequent_schools.merge(total_vacancies, on='vacancy_region_name')

most_frequent_schools.to_clipboard()

In [ ]:
# which regions have mean salaries higher than Rosstat 2024 average (58109)
mean_salary_by_region = filtered_df.groupby('vacancy_region_name')['vacancy_salary_min'].mean()
filter_regions = mean_salary_by_region[mean_salary_by_region >= 58109].index.tolist()
filtered_df[filtered_df['vacancy_region_name'].isin(filter_regions)]['vacancy_region_name'].value_counts()

In [ ]:
plt.figure(figsize=(8, 8))
sns.boxplot(data=df, x='personnel', y='salary_min')
plt.xlabel('Категория персонала')
plt.ylabel('Минимальная зарплата')
plt.title('Boxplot минимальной зарплаты по категории персонала')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
counts = df[df['personnel'] == 'старший']['region_name'].value_counts(ascending=False).head(10)
plt.figure(figsize=(14, 8))
counts.plot(kind='barh', color='skyblue')
plt.xlabel('Количество вакансий')
plt.ylabel('Регион')
plt.title('Топ-10 регионов по количеству вакансий для врачей акушеров-гинекологов')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
# Convert 'creation-date' to datetime and count vacancies per date
date_counts = pd.to_datetime(df['creation-date']).value_counts().sort_index()
plt.bar(date_counts.index, date_counts.values, width=5)
plt.xlabel('Дата создания вакансии')
plt.ylabel('Количество вакансий')
plt.title('Распределение вакансий по дате создания')
plt.tight_layout()
plt.show()

In [ ]:
# Setting the position of the bars on the y-axis
bar_width = 0.35  # Width of the bars

# Creating the bar chart
fig, ax = plt.subplots(figsize=(14, 20))

# Positions of the bars on the y-axis
y_positions = range(len(graph_data['region']))

# Plotting the bars
ax.barh(y_positions, graph_data['mean salary'], height=bar_width, color='blue', label='Росстат')
ax.barh([p + bar_width for p in y_positions], graph_data['mean salary RaRu'], height=bar_width, color='red', label='Работа в России')

# Adding labels
ax.set_yticks([p + bar_width / 2 for p in y_positions])
ax.set_yticklabels(graph_data['region'])
ax.set_xlabel('Средняя зарплата, тыс. руб.')
ax.set_title('Средняя зарплата учителей в январе-марте 2024 г. по регионам')
ax.legend()

# Inverting the y-axis to have the first region on top
plt.gca().invert_yaxis()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# # горизонтальная столбиковая диаграмма регионов по количеству вакансий в 2023 году
# most_vacancies.sort_values(ascending=True).plot(kind='barh', figsize=(10, 6), color='skyblue')
# plt.title('Топ-10 регионов по количеству вакансий в 2024 году')
# plt.ylabel('Регион')
# plt.xlabel('Кол-во вакансий')
# plt.show()

In [ ]:
data = filtered_df.loc[filtered_df['vacancy_salary_min'] != 0]
# data = data.loc[data['vacancy_creation-date'].dt.year >= 2023]

In [ ]:
graph_data = data['vacancy_salary_min']
g_weights = np.ones(len(graph_data))/len(graph_data)

fig, ax = plt.subplots(figsize=(15, 5))

# Creating the histogram with relatively thin bars
plt.hist(graph_data, bins=50, weights=g_weights, edgecolor = 'black')
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xlabel('Размер зарплаты')
plt.ylabel('Доля вакансий')
plt.title('Зарплаты учителей на портале trudvsem.ru')
plt.show()

In [ ]:
# # an iterative function to find a datapoint based on the name of the datapoint and the upper layer
# def recurs_find_key(key, obj): 
# 	if obj == None:
# 		return None
# 	else:
# 		if key in obj:
# 			return obj[key]
# 		if type(obj) == dict or type(obj) == list: 
# 			for k, v in obj.items(): 
# 				if type(v) == dict:
# 					result = recurs_find_key(key, v)
# 					return result
# 				elif type(v) == list:
# 					for el in range(len(v)):
# 						result = recurs_find_key(key, v[el-1])
# 						return result
